In [3]:
import json
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# -------------------- Step 1: 데이터 로드 및 전처리 (JSON 파일용) --------------------
# 'review.json' 파일을 올바르게 로드합니다.
try:
    # JSON 파일이 한 줄에 하나의 JSON 객체로 되어 있는 경우 (JSONL 형식)
    df = pd.read_json('review.json', lines=True)
    print("✅ JSON 파일 로드 성공.")
except FileNotFoundError:
    print("❌ Error: 'review.json' 파일이 존재하지 않습니다.")
    exit()
except ValueError as e:
    # JSON 파일이 단일 JSON 배열인 경우
    print(f"JSONL 형식이 아닌 것 같습니다. 일반 JSON 파일로 다시 시도합니다. (오류: {e})")
    try:
        df = pd.read_json('review.json')
        print("✅ 일반 JSON 파일 로드 성공.")
    except Exception as e:
        print(f"❌ Error: 'review.json' 파일을 읽는 데 실패했습니다. 파일 형식을 확인해주세요. (오류: {e})")
        exit()

# 필요한 컬럼만 선택하고, 결측치를 제거합니다.
df = df[['user_id', 'business_id', 'stars', 'text']].dropna()

# -------------------- 실험을 위한 함수 정의 --------------------
def run_experiment(df, random_seed):
    print(f"\n==================== 실험 시작: random_state={random_seed} ====================")
    
    # 데이터 분할
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=random_seed)

    # 훈련 세트의 고유한 사용자/아이템 ID를 기반으로 인덱스를 생성합니다.
    user2idx = {uid: i for i, uid in enumerate(train_df['user_id'].unique())}
    item2idx = {iid: i for i, iid in enumerate(train_df['business_id'].unique())}

    # 테스트 세트에만 존재하는 사용자/아이템을 위한 '알 수 없음' 인덱스를 추가합니다.
    unknown_user_idx = len(user2idx)
    unknown_item_idx = len(item2idx)

    # 사용자/아이템 ID를 인덱스로 매핑하고, '알 수 없음'은 새로운 인덱스로 채웁니다.
    train_df['user'] = train_df['user_id'].map(user2idx)
    train_df['item'] = train_df['business_id'].map(item2idx)
    test_df['user'] = test_df['user_id'].map(user2idx).fillna(unknown_user_idx).astype(int)
    test_df['item'] = test_df['business_id'].map(item2idx).fillna(unknown_item_idx).astype(int)

    # SBERT 문맥 벡터 생성
    print("SBERT 모델 로딩 및 문맥 벡터 생성 중...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sbert = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    train_context_vectors = sbert.encode(train_df['text'].tolist(), show_progress_bar=True)
    test_context_vectors = sbert.encode(test_df['text'].tolist(), show_progress_bar=True)

    # Dataset 클래스 정의
    class UCAMDataset(Dataset):
        def __init__(self, users, items, ratings, contexts):
            self.users = users
            self.items = items
            self.ratings = ratings
            self.contexts = contexts

        def __len__(self):
            return len(self.ratings)

        def __getitem__(self, idx):
            return (
                torch.tensor(self.users[idx], dtype=torch.long),
                torch.tensor(self.items[idx], dtype=torch.long),
                torch.tensor(self.contexts[idx], dtype=torch.float32),
                torch.tensor(self.ratings[idx], dtype=torch.float32)
            )

    # DataLoader 생성
    train_dataset = UCAMDataset(
        train_df['user'].values,
        train_df['item'].values,
        train_df['stars'].values.astype(np.float32),
        train_context_vectors
    )
    test_dataset = UCAMDataset(
        test_df['user'].values,
        test_df['item'].values,
        test_df['stars'].values.astype(np.float32),
        test_context_vectors
    )

    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

    # 모델 정의
    class UCAM(nn.Module):
        def __init__(self, num_users, num_items, context_dim=384, embed_dim=64):
            super().__init__()
            # '알 수 없음' 인덱스를 위해 +1
            self.user_embed = nn.Embedding(num_users + 1, embed_dim)
            self.item_embed = nn.Embedding(num_items + 1, embed_dim)
            self.fc_layers = nn.Sequential(
                nn.Linear(embed_dim * 2 + context_dim, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            )

        def forward(self, user_ids, item_ids, context_vecs):
            u = self.user_embed(user_ids)
            i = self.item_embed(item_ids)
            x = torch.cat([u, i, context_vecs], dim=-1)
            return self.fc_layers(x).squeeze()

    model = UCAM(num_users=len(user2idx), num_items=len(item2idx)).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    best_val_rmse = float('inf')
    epochs_no_improve = 0
    patience = 5
    min_delta = 0.001
    epochs = 50
    model_path = f'best_ucam_model_{random_seed}.pt'

    print("모델 학습 시작...")
    for epoch in range(epochs):
        model.train()
        total_train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} (Train)", leave=False)

        for user_ids, item_ids, context_vectors, stars in progress_bar:
            user_ids = user_ids.to(device)
            item_ids = item_ids.to(device)
            context_vectors = context_vectors.to(device)
            stars = stars.to(device)

            optimizer.zero_grad()
            predictions = model(user_ids, item_ids, context_vectors)
            loss = criterion(predictions, stars)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            
        # --- 검증 ---
        model.eval()
        val_preds, val_true = [], []
        with torch.no_grad():
            for user_ids, item_ids, context_vectors, stars in test_loader:
                user_ids = user_ids.to(device)
                item_ids = item_ids.to(device)
                context_vectors = context_vectors.to(device)
                stars = stars.to(device)

                preds = model(user_ids, item_ids, context_vectors)
                val_preds.extend(preds.cpu().numpy())
                val_true.extend(stars.cpu().numpy())

        val_preds = np.array(val_preds)
        val_true = np.array(val_true)
        val_mse = mean_squared_error(val_true, val_preds)
        val_rmse = np.sqrt(val_mse)
        val_mae = mean_absolute_error(val_true, val_preds)
        val_mape = np.mean(np.abs((val_true - val_preds) / (val_true + 1e-10))) * 100

        print(f"\nEpoch {epoch+1} | Train Loss: {total_train_loss / len(train_loader):.4f} | "
              f"Val RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}, MAPE: {val_mape:.2f}%")

        if val_rmse < best_val_rmse - min_delta:
            best_val_rmse = val_rmse
            epochs_no_improve = 0
            torch.save(model.state_dict(), model_path)
            print(f"  --> 개선됨. 모델 저장됨 (RMSE: {best_val_rmse:.4f})")
        else:
            epochs_no_improve += 1
            print(f"  --> 개선 없음. ({epochs_no_improve}/{patience})")
            if epochs_no_improve == patience:
                print("조기 종료 발생.")
                break
    
    # -------------------- 테스트셋 평가 --------------------
    def evaluate_model(model, data_loader, device):
        model.eval()
        preds, targets = [], []

        with torch.no_grad():
            for users, items, contexts, ratings in data_loader:
                users = users.to(device)
                items = items.to(device)
                contexts = contexts.to(device)
                ratings = ratings.to(device)

                output = model(users, items, contexts)
                preds.extend(output.cpu().numpy())
                targets.extend(ratings.cpu().numpy())

        preds = np.array(preds)
        targets = np.array(targets)

        mae = mean_absolute_error(targets, preds)
        mse = mean_squared_error(targets, preds)
        rmse = np.sqrt(mse)
        mape = np.mean(np.abs((targets - preds) / (targets + 1e-10))) * 100

        return mae, mse, rmse, mape

    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path))
        print(f"\n최적 모델 로드 완료: {model_path}")
        mae, mse, rmse, mape = evaluate_model(model, test_loader, device)

        print(f"\n✅ [UCAM] 최종 테스트 평가 지표 (random_state={random_seed}):")
        print(f"   - MSE  : {mse:.4f}")
        print(f"   - RMSE : {rmse:.4f}")
        print(f"   - MAE  : {mae:.4f}")
        print(f"   - MAPE : {mape:.2f}%")

        # MSE를 결과 딕셔너리에 추가
        return {'mse': mse, 'rmse': rmse, 'mae': mae, 'mape': mape}
    
    return None

# -------------------- Step 9: 5회 실험 및 평균 계산 --------------------
all_results = []
num_runs = 5
start_seed = 42

for i in range(num_runs):
    seed = start_seed + i
    results = run_experiment(df, seed)
    if results:
        all_results.append(results)

if all_results:
    avg_mse = np.mean([r['mse'] for r in all_results])
    avg_rmse = np.mean([r['rmse'] for r in all_results])
    avg_mae = np.mean([r['mae'] for r in all_results])
    avg_mape = np.mean([r['mape'] for r in all_results])

    print("\n\n==================== 5회 실험 평균 결과 ====================")
    print(f"✔️ 평균 MSE  : {avg_mse:.4f}")
    print(f"✔️ 평균 RMSE : {avg_rmse:.4f}")
    print(f"✔️ 평균 MAE  : {avg_mae:.4f}")
    print(f"✔️ 평균 MAPE : {avg_mape:.2f}%")
else:
    print("❌ 실험 결과를 얻지 못했습니다. 오류를 확인해주세요.")

✅ JSON 파일 로드 성공.

==================== 실험 시작: random_state=42 ====================
SBERT 모델 로딩 및 문맥 벡터 생성 중...


Batches: 100%|██████████| 2799/2799 [02:03<00:00, 22.58it/s]


모델 학습 시작...



Epoch 1 | Train Loss: 0.9286 | Val RMSE: 0.8269, MAE: 0.6573, MAPE: 24.01%
  --> 개선됨. 모델 저장됨 (RMSE: 0.8269)



Epoch 2 | Train Loss: 0.6534 | Val RMSE: 0.8148, MAE: 0.6410, MAPE: 24.00%
  --> 개선됨. 모델 저장됨 (RMSE: 0.8148)


...


Epoch 6 | Train Loss: 0.4661 | Val RMSE: 0.7716, MAE: 0.5975, MAPE: 21.23%
  --> 개선 없음. (1/5)



Epoch 7 | Train Loss: 0.4396 | Val RMSE: 0.7784, MAE: 0.6067, MAPE: 20.98%
  --> 개선 없음. (2/5)



Epoch 8 | Train Loss: 0.4142 | Val RMSE: 0.7792, MAE: 0.6048, MAPE: 21.32%
  --> 개선 없음. (3/5)



Epoch 9 | Train Loss: 0.3875 | Val RMSE: 0.7865, MAE: 0.6084, MAPE: 21.56%
  --> 개선 없음. (4/5)



Epoch 10 | Train Loss: 0.3609 | Val RMSE: 0.8004, MAE: 0.6182, MAPE: 22.09%
  --> 개선 없음. (5/5)
조기 종료 발생.

최적 모델 로드 완료: best_ucam_model_46.pt

✅ [UCAM] 최종 테스트 평가 지표 (random_state=46):
   - MSE  : 0.5894
   - RMSE : 0.7677
   - MAE  : 0.5948
   - MAPE : 21.24%


==================== 5회 실험 평균 결과 ====================
✔️ 평균 MSE  : 0.5943
✔️ 평균 RMSE : 0.7709
✔️ 평균 MAE  : 0.5951
✔️ 평균 MAPE : 21.32%
